# Polygenic models

Tutorial based on data from paper: https://elifesciences.org/articles/39702#s1

Paper evaluates polygenic models built on UK BioBank and GIANT GWAS for height
- Motivation: By adding sub-significant variants, we can explain more height variation.
- Cautions the use of high p-value thresholds as these variants likely capture population stratification

# Installation

In [ ]:
# Change to your individual
my_individual = 'NA20761'

In [ ]:
# Install Plink, download tutorial github
!wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20200921.zip -O plink.zip
!unzip plink.zip
!chmod +x plink
!git clone https://github.com/CCB293/Spring-2024.git

In [ ]:
# Import Python packages
import pandas as pd
import seaborn as sns
sns.set(style="ticks")
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
!pip install qqman
from qqman import qqman

# Download GWAS Statistics and Individual Info

## GWAS summary statistics from the paper

In [ ]:
# Downloading four polygenic risk scores for height, obtained from:
# https://github.com/msohail88/polygenic_selection/tree/master/polygenic_scores_pipeline
# The original GWAS summary statistics files used to develop the PRS are available at the following links:
# GIANT:
#   Consortium to "identify genetic loci that modulate human body size and shape"
#   253,000 individuals
#   https://portals.broadinstitute.org/collaboration/giant/index.php/GIANT_consortium_data_files#GIANT_Consortium_2012-2015_GWAS_Summary_Statistics
# UK Biobank
#   UK biorepository that investigates genetic predisposition for many phenotypes
#   336,000 individuals of British ancestry
#   Phenotype for standing height: https://docs.google.com/spreadsheets/d/1kvPoupSzsSFBNSztMzl04xMoSC3Kcx3CrjVf4yBmESU/edit#gid=227859291
!wget https://www.dropbox.com/sh/qi80hwjusnrt0nz/AABJ-lpSUa018qWWp3NSEVnia?dl=1 -O tutorial_files.zip
!unzip tutorial_files.zip

## Example of how to generate the above datasets

### Individual information

In [ ]:
# Individuals information
individuals = pd.read_csv('Spring-2024/week8/CCB293_Health_Inference_Tutorial/data/individuals.csv', '\t')
individuals.head()

In [ ]:
# Exercise: Pull your individual's information from dataframe
my_row = ...
my_super_population = my_row.super_population.values[0]
my_population = my_row.population.values[0]
print("My individual: %s; Population: %s;  Superpopulation %s" % (my_individual, my_population, my_super_population))

### GWAS summary statistics

In [ ]:
# Download GIANT Height GWAS statistics (p-values, effect sizes)
!wget https://portals.broadinstitute.org/collaboration/giant/images/0/01/GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.txt.gz

In [ ]:
# How is this file formatted?
# Allele1: effect allele
# Allele2: non-effect allele
# Freq.Allele1.HapMapCEU: allele frequency of effect allele in GWAS population
# b: effect size (log-odds ratio per copy of effect allele)
# SE: standard error in effect size
# p: p-value (used for filtering)
# N: number of individuals in study
!zcat GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.txt.gz | head

In [ ]:
# Reformat header of GWAS statistic file
!gunzip GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.txt.gz
!head GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.txt
!sed "1d" GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.txt > GIANT_height.txt
!sed -i '1i SNP Allele1	Allele2	Freq.Allele1.HapMapCEU	b	SE	P	N' GIANT_height.txt
!head GIANT_height.txt

In [ ]:
# Exercise: How many variants are in this file?

### Form Polygenic Models

In [ ]:
# Forming our polygenic model
# Clumping and thresholding works by iteratively looking at your lowest p-value
#   sites, include that lead variant and removing any variants in a window (clump)
#   that are correlated. Stop when you reach your P-value threshold
# --vcf: input genetic data as vcf
# --clump: Clumping and thresholding PRS method
# --clump-p1: 0.01 P-value threshold
# --clump-r2: Remove variants with >10% correlation (LD)
# --clump-kb: 1MB region to prune variants in LD
!./plink --vcf 1KG_snps_subsetted.vcf.gz \
        --clump GIANT_height.txt \
        --clump-p1 0.01 \
        --clump-r2 0.1 \
        --clump-kb 1000 \
        --out GIANT_height

In [ ]:
# Visualize weights of polygenic score
filtered = pd.read_csv('GIANT_height.clumped', delim_whitespace=True)[["SNP", "CHR", "BP"]]
print(filtered.shape)
height = pd.read_csv('GIANT_height.txt', delim_whitespace=True)
merged = pd.merge(filtered, height, on="SNP")
merged

## GWAS summary statistics from the paper

In [ ]:
# Read GWAS Polygenic model from the GIANT study (p-value threshold 0.01) 253,000 individuals
pd.read_csv('GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.header.txt.clumpedout.0.01', delim_whitespace=True)

In [ ]:
# Read GIANT Polygenic model (p-value threshold 5E-8)
pd.read_csv('GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.header.txt.clumpedout.5E-8', delim_whitespace=True)

In [ ]:
# Read UK BioBank Polygenic model score (p-value threshold 0.01)
# tstat: t-statistic
pd.read_csv('50.assoc.tsv.processed.nodups.clumpedout.0.01', delim_whitespace=True)

In [ ]:
# Read UK BioBank Polygenic model score (p-value threshold 5e-8)
# tstat: t-statistic
pd.read_csv('50.assoc.tsv.processed.nodups.clumpedout.5E-8', delim_whitespace=True)

In [ ]:
# Exercise: what is the difference between the four score files? Do the number of SNPs in each score file make sense?

# Scoring Individuals with Polygenic Models

In [ ]:
# Scoring each individual
# ./plink --vcf <vcf_file>
#         --score <GWAS summary statistics> <snp_id column> <effect allele column>
#                 <effect size column> <file_includes_header> <sum or average>
#         --out <out_directory>
# https://www.cog-genomics.org/plink/1.9/score
!./plink --vcf 1KG_snps_subsetted.vcf.gz \
         --score GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.header.txt.clumpedout.0.01 1 2 5 header sum \
         --out giant_prs_0.01

In [ ]:
# Score each individual for GIANT 5E-8
!./plink --vcf 1KG_snps_subsetted.vcf.gz \
         --score GIANT_HEIGHT_Wood_et_al_2014_publicrelease_HapMapCeuFreq.header.txt.clumpedout.5E-8 1 2 5 header sum \
         --out giant_prs_5E-8

In [ ]:
# Score each individual for UK BioBank 0.01
!./plink --vcf 1KG_snps_subsetted.vcf.gz \
         --score 50.assoc.tsv.processed.nodups.clumpedout.0.01 1 2 5 header sum \
         --out 50_assoc_0.01

In [ ]:
# Score each individual for UK BioBank 5E-8
!./plink --vcf 1KG_snps_subsetted.vcf.gz \
          --score 50.assoc.tsv.processed.nodups.clumpedout.5E-8 1 2 5 header sum \
          --out 50_assoc_5E-8

In [ ]:
# Output score for each individual
# PHENO = phenotype if available, -9 if not available
# CNT = total allele count
# CNT2 = number of effect alleles
# SCORESUM = polygenic score
pd.read_csv('giant_prs_0.01.profile', delim_whitespace=True)

# Plotting Polygenic Model Results

In [ ]:
# Loading results from the four scores into one DataFrame to plot results easily

def load_results(name):
    results = pd.read_csv(name+'.profile', delim_whitespace=True).merge(individuals, left_on='FID', right_on='individual')
    results['method'] = name
    # Standardizing the polygenic score
    results['standardized_score'] = (results.SCORESUM - results.SCORESUM.mean()) / results.SCORESUM.std()
    results['standardized_score_EUR'] = (results.SCORESUM - results.SCORESUM[results.super_population=='EUR'].mean()) / results.SCORESUM[results.super_population=='EUR'].std()
    results['standardized_score_my_super_population'] = \
                    (results.SCORESUM - results.SCORESUM[results.super_population==my_super_population].mean()) / results.SCORESUM[results.super_population==my_super_population].std()
    return results

results = pd.concat([load_results(study) for study in ['giant_prs_0.01', 'giant_prs_5E-8', '50_assoc_0.01', '50_assoc_5E-8']])

In [ ]:
# Plot of GIANT 0.01 PRS
sns.boxplot(x="super_population", y="standardized_score",
            data=results[results["method"]=="giant_prs_0.01"], color="b");
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);
plt.title("GIANT 0.01 PRS");

In [ ]:
# Comparing scores across continental groups
sns.boxplot(x="super_population", y="standardized_score",
            hue="method",
            data=results);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [ ]:
# Plotting scores on the European populations
sns.boxplot(x="population", y="standardized_score_EUR",
            hue="method",
            data=results[results.super_population=='EUR']);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [ ]:
# Plot Q-Q for GIANT
df = pd.read_csv("GIANT_height.txt", delim_whitespace=True)
qqman.qqplot(df[["P"]], show=True)

In [ ]:
# Display figure from paper: https://elifesciences.org/articles/39702#s1
# Shows evidence for population stratification in GIANT dataset
Image("https://iiif.elifesciences.org/lax:39702%2Felife-39702-fig2-v2.tif/full/1500,/0/default.jpg")

In [ ]:
# Plotting scores on my continental group with my individual
results.population[results.individual==my_individual] = 'My individual'
sns.boxplot(x="population", y="standardized_score_my_super_population",
            hue="method",
            data=results[results.super_population.isin([my_super_population, 'My individual'])]);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [ ]:
# Plotting scores on my population
sns.boxplot(x="population", y="standardized_score_my_super_population",
            hue="method",
            data=results[results.population.isin([my_population, 'My individual'])]);
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);

In [ ]:
# Exercise: How consistent are the standardized scores for your individual?

Takeaways
- P-value threshold is important for polygenic scores
- Low p-value variants might capture population stratification
- Validation on held-out data